
# Occupany Grid Mapping

This problem of estimating the full state of a 100 x 100 world can be treated as a Dynamic Occupancy Grid Mapping. The assumption that data is noise-free makes it possible to use staright forward startegies:
*  Last Best ValueHit/Miss counter 

For a noiseless setting, the sensor model will look like this:
>$P(Z| Occlusion) = 1$             ,sensor only gives a measurement when an obstacle is detected

>$P(Z| !Occlusion) = 0$            ,no measurments for empty space


For a noisy setting, a sensor model might look like this:

>$P(Z| Occlusion) = 0.8$            ,80% of the times an obstacle is detected, sensor gives a reading

>$P(Z| !Occlusion) = 0.3$           ,30% of the times in ama epmty space, sensor gives false readings

In [ ]:
import numpy as np

In [3]:
def findPosterior(P_Occ, P_Z_Occ, P_Z_NOcc):
    P_Z = (P_Z_Occ*P_Occ) + (P_Z_NOcc*(1- P_Occ))
    P_Occ_Z = (P_Z_Occ*P_Occ) / P_Z
    return P_Occ_Z

In [10]:
print findPosterior(0.3, 0.2, 1)

0.0789473684211


In [26]:
Observations = {'hit', 'miss', 'no_obs'};
States = {'Occluded', 'Empty'}

## Observation Matrix, Bz
3 Matrices which describe the observation model. Noise can be added here. This matrix is pre-decided and can be applied to every cell of the grid.  

$$\mathbf{B_z(h)} = \left[\begin{array}{rc}
p(h | Occ) & 0\\
0 & p(h | Empty) \\
\end{array}\right]$$

$$\mathbf{h \in (hit, miss, no\_obs)}$$

In [20]:
import numpy as np
B_z = {state : np.zeros((2,2)) for observation in Observations}
print B_z

{'no_obs': array([[ 0.,  0.],
       [ 0.,  0.]]), 'miss': array([[ 0.,  0.],
       [ 0.,  0.]]), 'hit': array([[ 0.,  0.],
       [ 0.,  0.]])}


In [22]:
B_z['hit'][0, 0] = 1;
B_z['miss'][1, 1] = 1;
B_z['no_obs'][0, 0] = 0.01;
B_z['no_obs'][1,1] = 0.01;

In [23]:
print B_z

{'no_obs': array([[ 0.01,  0.  ],
       [ 0.  ,  0.01]]), 'miss': array([[ 0.,  0.],
       [ 0.,  1.]]), 'hit': array([[ 1.,  0.],
       [ 0.,  0.]])}


## Cell State Transition Matrix, Ac
A 2x2 matrix that we "learn" overtime. This matrix controls how the state of a cell changes over a time step. We update this matrix after every time step.

The Cell state transforms as follows: $p(c_t | c_{t-1})$
$$\mathbf{A_c} = \left[\begin{array}{rc}
p(Occ | Occ) & p(Empty | Occ)\\
p(Occ | Empty) & p(Empty | Empty) \\
\end{array}\right]$$

In [29]:
# We need to add this to every cell. We keep a history of past points
transition_parameters = {}; ## Dictionary of 12 probability values per time step
for observation in Observations:
    print observation
    state_prob = {};  #Dictionary to hold Cell State Transition Probabilities i->j
    for state_i in States: # State at time t-1
        for state_j in States: # State at time t 
            state_prob[state_i+'_'+state_j] = []; # List holds past probabilities
    transition_parameters[observation] = state_prob;

print transition_parameters


no_obs
hit
miss
{'no_obs': {'Occluded_Empty': [], 'Occluded_Occluded': [], 'Empty_Empty': [], 'Empty_Occluded': []}, 'miss': {'Occluded_Empty': [], 'Occluded_Occluded': [], 'Empty_Empty': [], 'Empty_Occluded': []}, 'hit': {'Occluded_Empty': [], 'Occluded_Occluded': [], 'Empty_Empty': [], 'Empty_Occluded': []}}


In [37]:
# Calculating transition parameters
time_steps = 1
A_c_prob = {}
for state_i in States: # State at time t-1
    for state_j in States: # State at time t 
        for observation in Observations:
            p = np.sum(transition_parameters[observation][state_i+'_'+state_j]);
            p /= time_steps;
            A_c_prob[state_i+'_'+state_j] = p;

time_steps += 1;

A_c = np.zeros((2,2));

A_c[0, 0] = A_c_prob['Occluded_Occluded'];
A_c[0, 1] = A_c_prob['Empty_Occluded'];
A_c[1, 0] = A_c_prob['Occluded_Empty'];
A_c[1, 1] = A_c_prob['Empty_Empty'];
print A_c
#need to normalise
norm = max(A_c[0, 0] + A_c[0, 1], 0.0000001)
A_c[0, 0] /= norm
A_c[0, 1] /= norm

norm = max(A_c[1, 0] + A_c[1, 1], 0.0000001)
A_c[1, 0] /= norm
A_c[1, 1] /= norm

[[ 0.  0.]
 [ 0.  0.]]


## Cell State Matrix, Q
A 1x2 matrix that shows the state of the cell. This is updated for each cell at every time step. 

The update equation is as follows: $Q_t = Q_{t-1} A_c B_z \eta$
$$\mathbf{Q} = \left[\begin{array}{rc}
p(Occ | z_{1:t}) & p(Empty | z_{1:t})\\
\end{array}\right]$$

In [38]:
Q = np.zeros((1, 2))

In [ ]:
##init
#A_c = [0.9 0.1
#       0.1 0.9]
#Q = [0.9 1] //Assuming all cells are occupied in beginning..also safer

#FOR every time step
#read a cell value from the scan
# A = A_c 
# if scan_value == 0
# observation = 'no_obs'
## B = B_z[observation]

#read the state from grid map
# state_i = Occluded or Empty
def update(Q, A, B):
    newQ = Q.dot(A.dot(B));
    norm = newQ.sum();
    newQ /= norm;
    return newQ

Q = update(Q, A, B)

if (Q[0] > 0.5): ##Occupied
    #set cell value to 255 in map
    #state_j = "Occupied"
    p = Q[0]
    print "Occupied"

else:##Empty
    #set cell value to 0 in map
    #state_j = "Empty""
    p = 1- Q[0]
    print "Empty"

transition_parameters[observation][state_i + '_' + state_j].append(p);
    
        
